In [1]:
# Leer datos del proyecto 

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/home/leonardo/Documentos/ITAM/PCA-nutricion/data/nndb_flat.csv', encoding = "L1")

In [4]:
df.shape

(8618, 45)

In [5]:
datos_limpios = df

In [6]:
datos_limpios.drop(datos_limpios.columns[datos_limpios.columns.str.contains('_USRDA')].values, 
        inplace=True, axis=1)

In [7]:
numericas = datos_limpios.drop(columns=['ScientificName','ID','FoodGroup','ShortDescrip','Descrip','CommonName','MfgName'])

In [8]:
numericas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 23 columns):
Energy_kcal      8618 non-null int64
Protein_g        8618 non-null float64
Fat_g            8618 non-null float64
Carb_g           8618 non-null float64
Sugar_g          8618 non-null float64
Fiber_g          8618 non-null float64
VitA_mcg         8618 non-null int64
VitB6_mg         8618 non-null float64
VitB12_mcg       8618 non-null float64
VitC_mg          8618 non-null float64
VitE_mg          8618 non-null float64
Folate_mcg       8618 non-null int64
Niacin_mg        8618 non-null float64
Riboflavin_mg    8618 non-null float64
Thiamin_mg       8618 non-null float64
Calcium_mg       8618 non-null int64
Copper_mcg       8618 non-null float64
Iron_mg          8618 non-null float64
Magnesium_mg     8618 non-null int64
Manganese_mg     8618 non-null float64
Phosphorus_mg    8618 non-null int64
Selenium_mcg     8618 non-null float64
Zinc_mg          8618 non-null float64
dtypes: 

In [9]:
numericas.head()

,Energy_kcal,Protein_g,Fat_g,Carb_g,Sugar_g,Fiber_g,VitA_mcg,VitB6_mg,VitB12_mcg,VitC_mg,...,Riboflavin_mg,Thiamin_mg,Calcium_mg,Copper_mcg,Iron_mg,Magnesium_mg,Manganese_mg,Phosphorus_mg,Selenium_mcg,Zinc_mg
0,717,0.85,81.11,0.06,0.06,0.0,684,0.003,0.17,0.0,...,0.034,0.005,24,0.000,0.02,2,0.000,24,1.0,0.09
1,717,0.85,81.11,0.06,0.06,0.0,684,0.003,0.13,0.0,...,0.034,0.005,24,0.016,0.16,2,0.004,23,1.0,0.05
2,876,0.28,99.48,0.00,0.00,0.0,840,0.001,0.01,0.0,...,0.005,0.001,4,0.001,0.00,0,0.000,3,0.0,0.01
3,353,21.40,28.74,2.34,0.50,0.0,198,0.166,1.22,0.0,...,0.382,0.029,528,0.040,0.31,23,0.009,387,14.5,2.66
4,371,23.24,29.68,2.79,0.51,0.0,292,0.065,1.26,0.0,...,0.351,0.014,674,0.024,0.43,24,0.012,451,14.5,2.60


Escalar las variables

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
df_nutrientes = scaler.fit_transform(numericas)

Funciones del examen anterior que calculan factorización QR

In [12]:
import numpy as np
import copy
import codecs
import sys
from scipy.linalg import solve_triangular

def busca_ayuda(cadena):
    """
    Función que devuelve el texto de ayuda correspondiente a la función
    que se pase como parámetro, obtenida de un unico archivo .txt
    donde están documentadas las "ayudas" de todas las funciones asociadas
    a la factorización QR (contenidas en este .py)
    
    params: cadena  nombre de la función de la que se desea obtener ayuda
                    
    return: help    ayuda de la función buscada en forma de texto
    
    """
    l=len(cadena)
    help=codecs.open('Help_funciones_factorizacion_QR.txt', "r", "utf-8").read()
    p_inicio=help.find("****i****" + cadena)
    p_final=help.find("****f****" + cadena)
    help=help[(p_inicio+9+l):p_final]
    return help






def crear_matriz_aleatoria(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
    params: renglones       no. de renglones de la matriz
            columnas        no. de columnas de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if isinstance(renglones, int)==False or isinstance(columnas, int)==False:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')
    elif renglones<=0 or columnas<=0:
        sys.exit('Los parámetros de renglones y columnas deben ser enteros positivos')            
    if isinstance(maximo_valor, (int, float))==False or isinstance(minimo_valor, (int, float))==False:
        sys.exit('Los parámetros maximo_valor y minimo_valor deben ser numericos')

    #Se inicializa una matriz llena de ceros con las dimensiones deseadas (mxn)
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            #Si entero es verdadero se obtiene el maximo entero menor o igual a 1 (//1)
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

#crear_matriz_aleatoria.__doc__ =busca_ayuda("crear_matriz_aleatoria")






def house(x):
    """
    Función que calcula vector de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    factor para obtener matriz de reflexión householder Rf
            v       vector de householder
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(x) is not np.ndarray:
        sys.exit('x debe ser de tipo numpy.ndarray')
    
    #longitud del vector x=(x_0,x_1,x_2,...,x_(m-1))
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    #Se hace v=x=(1,x_1,x_2,...,x_(m-1))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    #con las siguientes condiciones se checa si x es múltiplo del vector canónico e_1 
    #y el signo de x[0]
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

#house.__doc__ =busca_ayuda("house")






def matriz_auxiliar_Arv(A,ind_singular=False):
    """
    Función que genera una matriz que contiene los elementos r distintos de cero de la 
    matriz R y las entradas de los vectores householder v (excepto la primera), con los 
    cuales se puede calcular la matriz Q. Ambas matrices componentes de la factorización QR
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: Arv    Matriz (mxn) que incluye las componentes distintas de cero de la matriz R 
                   y los vectores householder con los que se puede obtener la matriz Q, y 
                   con ello la factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    m,n=A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    
    #m contiene el numero de renglones y n el de columnas
    m,n=A.shape
    #se crea una matriz con los valores de A
    Arv=copy.copy(A)
    
    #si no se pide que se cheque que la matriz es singular, se obtendrá la matriz R
    #aún si esta es singula (que exista algún elemento de la diagonal que sea 0
    if ind_singular==False:
        for j in range(n):
            beta, v=house(Arv[j:m,j])
            #Con esta multiplicación se van generando las componentes r de la matriz R
            Arv[j:m,j:n]=Arv[j:m,j:n]-beta*np.outer(v,(np.transpose(v)@Arv[j:m,j:n]))
            #se guarda en cada columnas los valores de v d, excepto la primer componente (que vale 1)
            Arv[(j+1):m,j]=v[1:(m-j)]
    else:
        for j in range(n):
            beta, v=house(Arv[j:m,j])
            Arv[j:m,j:n]=Arv[j:m,j:n]-beta*np.outer(v,(np.transpose(v)@Arv[j:m,j:n]))
            #Si se detecta que alguna entrada de la diagonal (elemento de la maatriz R)
            #es cero, se devuelve un valor nulo (None), y para el cálculo de Arv
            #No se condiciona sobre si alcanza el valor de cero, sino más bien
            #si está lo suficientemente cercano, pues dados los errores por redondeo
            #no siempre se alcanza el cero aunque teóricamente sí lo sea, pero el
            #algoritmos arroja un valor cercano a cero
            if Arv[j,j]<10**(-14) and Arv[j,j]>-10**(-14):
                return None
            Arv[(j+1):m,j]=v[1:(m-j)]
    return Arv

#matriz_auxiliar_Arv.__doc__ =busca_ayuda("matriz_auxiliar_Arv")






def matriz_Q_R(A,ind_singular=False):
    """
    Función que devuelve la matriz R y Q de la factorización QR de una matriz A
    
    params: A    Matriz (mxn)
    return: Q    Matriz Q (mxm) de la factorización A=QR
            R    Matriz Q (mxm) de la factorización A=QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray:
        sys.exit('A debe ser de tipo numpy.ndarray')
    elif A.shape[0]<A.shape[1]:
        sys.exit('El numero de renglones de A tiene que ser igual o mayor al no. de columnas')
    
    #si ind_singular es True, entonces no se devuelve la matriz R ni Q
    #esto se plantea así como un sistema de contról, para evitar comprobar que las matrices
    #sean singulares (y no tengan una única solución) mediante el determinante, que
    #involucra un alto costo computacional para matrices de tamaño representativo
    Arv=matriz_auxiliar_Arv(A,ind_singular)

    if Arv is None:
        return None, None
    else:
        m,n=A.shape
        Q=np.eye(m)
        R=copy.copy(A)
        for j in range(n):
            Qj=Q_j(Arv,j+1)
            Q=Q@Qj
            R=Q_j(Arv,j+1)@R
        return Q,R

#matriz_Q_R.__doc__ =busca_ayuda("matriz_Q_R")






def Q_j(Arv,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van 
    obteniendo n Qj's, que si se multiplican todas da por resultado Q=Q_1*Q_2*...*Q_n)
                            
    params: Arv   Matriz (mxn) con la info escencial
            j     indica el índice de la matriz Q_j
    return: Qj    Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(Arv) is not np.ndarray:
        sys.exit('Arv debe ser de tipo numpy.ndarray')
    elif Arv.shape[0]<Arv.shape[1]:
        sys.exit('El numero de renglones de Arv tiene que ser igual o mayor al no. de columnas')
    if isinstance(j, int)==False:
        sys.exit('El parámetro j deber ser un entero')
    elif 1>j or j>Arv.shape[1]:
        sys.exit('El parámetro j debe estar en el rango [1,no. columnas de Arv]')
    
    m,n=Arv.shape
    Qj=np.eye(m)
    #Para construir Q_j requerimos usar el vector v contenido en Arv contenido
    #en la j-1 columna (considerando que en Python la primer columna es la cero)
    v=np.concatenate((1,Arv[j:m,(j-1)]), axis=None)
    beta=2/(1+Arv[j:m,(j-1)].dot(Arv[j:m,(j-1)]))
    Qj[(j-1):m,(j-1):m]=np.eye(m-(j-1))-beta*np.outer(v,v)
    return Qj

#Q_j.__doc__ =busca_ayuda("Q_j")






def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineales (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    return: x   vector que satisface (Ax=b)
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones y 1 columna")

    #Si la matriz A es singular, Q y R serán igual a None
    Q,R=matriz_Q_R(A,1)
    if Q is None:
        sys.exit('La matriz A asociada al sistema de ecuaciones es singular')
    b_prima=np.transpose(Q)@b
    x=solve_triangular(R, b_prima,lower=False)
    return x

#Solucion_SEL_QR_nxn.__doc__ =busca_ayuda("Solucion_SEL_QR_nxn")





def crear_bloques(A, b, m1=False, n1=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            m1  Numero de columnas que tendrá el 1er bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones y 1 columna")    
    
    if m1==False:
        m1=m//2
    if n1==False:
        n1=n//2
    
    b1 = b[:m1]
    b2 = b[m1:m]
    A11 = A[:m1,:n1]
    A21 = A[m1:m,:n1]
    A12 = A[:m1,n1:n]
    A22 = A[m1:m,n1:n]

    return A11,A21,A12,A22,b1,b2

#crear_bloques.__doc__ =busca_ayuda("crear_bloques")






def eliminacion_bloques(A,b, m1=False, n1=False):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (mxn) que representa los coeficientas de las ecuaciones
            b   vector (mx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    
    #Se checa que los parámetros sean congruentes con la funcionalidad
    if type(A) is not np.ndarray or type(b) is not np.ndarray: #esto implica que A y b tienen más de 1 elemento
        sys.exit('A y b deben ser de tipo numpy.ndarray')
    m,n = A.shape
    if m<n:
        sys.exit('EL numero de renglones de A debe ser mayor o igual al no. de columnas')
        
    elif b.shape[0]!= m:
        sys.exit('b debe representar un vector columna (mx1), m=no. renglones de A')
    else:
        try:
            flag=b.shape[1]
        except Exception:
            flag=1
        if flag!=1:
            sys.exit("b debe representar un vector de m renglones y 1 columna")
    
    if m1==False:
        m1=m//2
    if n1==False:
        n1=n//2

    if m != n:
        sys.exit('A debe ser cuadrada')

#     if np.linalg.det(A)==0:
#         sys.exit('A debe ser no singular')

    A11,A21,A12,A22,b1,b2=crear_bloques(A,b,m1,n1)

    # 1. Calcular Y=A11^{-1}A12 y y=A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    #if np.linalg.det(A11)==0:
    #    sys.exit('A11 debe ser no singular')
    y=Solucion_SEL_QR_nxn(A11,b1)
    
    Y=np.zeros((n1,n-n1))
    for j in range(n-n1):
        Y[:,j]=Solucion_SEL_QR_nxn(A11,A12[:,j])
    
    # 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S=A22-A21@Y
    b_h=b2-A21@y

    # 3. Resolver Sx2 = b_hat
    x2=Solucion_SEL_QR_nxn(S,b_h)

    # 4. Resolver A11x1 = b1-A12x2
    x1=Solucion_SEL_QR_nxn(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

#eliminacion_bloques.__doc__ =busca_ayuda("eliminacion_bloques")

## Funcion creada para este proyecto final

Función que ocupa algoritm QR para calcular los eigenvectores de una matriz

In [13]:
def eigenvectores_eigenvalores_QR_vf(data,niter,tolerancia = 10**-8):
    """
    Función para obtener los eigenvectores y eigenvalores de una matriz cualquiera
    params: data			matriz de datos
            niter: número de iteraciones máximas 
    
    
    return:     eigenvalores 
                eigenvalores 
                
    Depende de las funciones ue calculan la factorización QR
    """

    # convertir a array
    A = np.array(data)
    
    # La matriz de covarianza ya es simetrica, no necesitamos rellenar columnas de ceros
    # Completamos la matriz A con columnas de ceros para que sea cuadrada
    n, p = A.shape
    columnas = n - p
    ceros = np.zeros((n,columnas))
    
    # Matriz inicial
    A0 = np.append(A, ceros, axis = 1)
    # Factorización inicial
    Qk,Rk =matriz_Q_R(A0)
    
    # Hacer una copia de la matriz Q inicial
    Q = Qk.copy()
    
    valor = 10
    
    #iniciamos ciclo
    for i in range(niter):
        
        #Definimos la nueva Ak+1
        Ak = Rk@Qk
        #calculamos la factorización QR sobre la nueva Ak
        Qk,Rk =matriz_Q_R(Ak)
        
        # Actualizar la matriz Q
        Q = Q@Qk
        
        
        # Se usan los valores distintos de la diagonal para corroborar la tolerancia deseada
        valor = np.sum(np.abs(Qk)) - np.trace(np.abs(Qk))
        
        
        # Detener si se logra la tolerancia deseada
        # Notar que en esta primera prueba no se está actualizando  el valor que checa la tolerancia
        if (valor<tolerancia):
            print("se alcanzo la torelacia deseada")
            break
    
    #se acaba el for
    
    #La diagonal de Ak nos regresa los eigenvalores
    eigenvalores = np.diagonal(Ak)
        
    return eigenvalores, Q

### Probar la función

Crear una matriz de covarianzas

In [14]:
mean_vec = np.mean(df_nutrientes, axis=0)

# Matriz de covarianzas
C = (df_nutrientes - mean_vec).T.dot((df_nutrientes - mean_vec)) / (df_nutrientes.shape[0]-1)
C

array([[ 1.00011605e+00,  1.10314826e-01,  8.06888349e-01,
         4.87152820e-01,  3.13187760e-01,  1.97360607e-01,
         2.63319240e-02,  1.21886189e-01, -1.22936357e-02,
        -3.34986997e-02,  3.03736260e-01,  1.45953262e-01,
         1.75011508e-01,  1.55848551e-01,  1.87460547e-01,
         1.23585367e-01,  9.95244299e-02,  1.95587000e-01,
         2.55909273e-01,  3.76253802e-02,  1.95203296e-01,
         5.66793206e-02,  1.12779439e-01],
       [ 1.10314826e-01,  1.00011605e+00,  5.47217340e-02,
        -3.02008975e-01, -2.66200429e-01, -7.27858501e-02,
         2.64858692e-02,  2.28511835e-01,  2.45451882e-01,
        -6.64632916e-02, -2.94848805e-02,  8.94030958e-03,
         3.76983043e-01,  2.02503229e-01,  9.83711284e-02,
         4.68937280e-02,  1.58918049e-01,  1.26389996e-01,
         2.18635959e-01,  3.97470507e-02,  4.44659202e-01,
         3.78413082e-01,  4.16363691e-01],
       [ 8.06888349e-01,  5.47217340e-02,  1.00011605e+00,
        -5.41224131e-02, -1.9

In [15]:
e, q = eigenvectores_eigenvalores_QR_vf(C,10)

In [16]:
print(e)
print('--------------------------------------------------------------------------')
print(q)

[5.44991441e+00 2.61797014e+00 2.02972424e+00 1.65005670e+00
 1.86641120e+00 1.13445534e+00 1.06408339e+00 8.68331083e-01
 8.60156528e-01 8.86362763e-01 7.33685165e-01 5.67458181e-01
 5.00363037e-01 5.02198917e-01 3.33008339e-01 4.08115249e-01
 3.32553367e-01 2.66385369e-01 3.18388529e-01 2.38233674e-01
 2.11195400e-01 1.59822432e-01 3.79568767e-03]
--------------------------------------------------------------------------
[[ 1.57990336e-01 -2.84447153e-01  4.73895575e-01 -2.42831806e-01
   1.25131302e-01  1.01450256e-01 -8.80435214e-02 -6.60609299e-02
  -9.11277861e-02  1.30668059e-01 -1.43251953e-01 -1.13459083e-01
  -6.27872311e-02  3.79243323e-02  1.08875745e-01  3.05480334e-02
   7.12927356e-02 -6.96291630e-02 -5.29083030e-03  8.49527843e-02
  -1.30373514e-01 -4.13090282e-02  6.78011231e-01]
 [ 1.40443482e-01  3.43892880e-01  2.24706618e-01 -5.79863640e-02
  -2.96084603e-01  1.42178628e-01 -5.81675610e-02  6.63767504e-02
  -2.01389902e-01  4.66520097e-02 -2.77276177e-01 -2.9117104

Usar numpy para cacular los valores y eigen vectores de una matriz de covarianzas

In [17]:
eigenvalores, eigenvectotes= np.linalg.eig(C)

In [18]:
print(eigenvalores)
print("----------------------------------------------")
print(eigenvectotes)

[5.44991815e+00 2.61876222e+00 2.03213339e+00 1.87934951e+00
 1.63586122e+00 1.14050300e+00 1.06099038e+00 3.79568767e-03
 9.26361287e-01 8.62118205e-01 8.24782146e-01 7.31317415e-01
 5.96964233e-01 5.08712086e-01 4.69516142e-01 1.59112142e-01
 4.08033662e-01 2.11183417e-01 2.38166939e-01 2.56232135e-01
 3.37973220e-01 3.29606937e-01 3.21275605e-01]
----------------------------------------------
[[ 1.57813630e-01 -2.73448664e-01  4.62006190e-01  5.22804571e-02
   2.93596273e-01  1.32865473e-01 -6.68230864e-02 -6.78011231e-01
  -7.44071538e-02  4.15483417e-02 -1.41090875e-01 -1.40246655e-01
   1.17138709e-01 -5.76914014e-02 -6.69417145e-02 -3.80912465e-02
   3.91931601e-02  1.29039040e-01 -7.63316662e-02  7.83573294e-02
   4.52270825e-02  1.00240632e-01  6.36675416e-02]
 [ 1.40619902e-01  3.43396770e-01  2.13566991e-01 -3.11112051e-01
  -1.31758602e-02  1.41468918e-01 -6.29942500e-03  1.78929900e-01
  -1.16308539e-01  1.48471168e-01 -1.83555382e-02 -3.07117422e-01
  -1.29540212e-01 -3.1

In [19]:
def PCA_from_QR_vf(data,niter):
    """
    Función para PCA a partir de los eigenvectores  
    params: data:			matriz de datos
            niter:      número de iteraciones máximas    
    
    
    return:     componentes		Los coeficientes para calcular los componentes principales (eigenvectores de la matriz de covarianzas)
                Z			Los datos transformados (componentes principales)
                varianza_explicada	La varianza explicada por cada componente principal
    
    Depende de la función: eigenvectores_QR
    """
    
    # convertir a array
    A = np.array(data)
    
    # Centrar los datos
    mean_vec = np.mean(A, axis=0)
    datos_centrados = (A - mean_vec)

    # Matriz de Covarianzas
    #C = (datos_centrados.T@datos_centrados)/(datos_centrados.shape[0]-1)
    C = (A - mean_vec).T.dot((A - mean_vec)) / (A.shape[0]-1)
    
    # Calcular algoritmo QR
    E, Q = eigenvectores_eigenvalores_QR_vf(C,niter)
    
    
    # Los componentes (coeficientes)
    componentes = Q.T
    
    # Los datos transformados (componentes principales)
    # Aquí marcaba error al filtrar porque no se reconocia a Z como numpy array
    Z = datos_centrados@Q
    
    # La varianza explicada
    varianza_explicada = E/np.sum(E)
    
    # Calcula número de componentes de manera automatica de acuerdo a la variana explicada
    # Threshold de 60%
    n = data.shape[1] #numero de columnas
    varianza_acumulada = varianza_explicada.cumsum()
    conteo = (varianza_acumulada)  <  0.6
    num_componentes = conteo.sum() + 1 
    
    # regresar 4 objetos
    return E[:num_componentes], componentes[:num_componentes].T, Z[:,:num_componentes], varianza_explicada[:num_componentes] #, varianza_acumulada, num_componentes
    #return E, componentes, Z, varianza_explicada, varianza_acumulada, num_componentes

### Probar función

In [20]:
eigenvalores, coeficientes, Z, varianza_explicada = PCA_from_QR_vf(df_nutrientes,380)

In [21]:
eigenvalores

array([5.44991815, 2.61876222, 2.03213339, 1.87934951, 1.63586122,
       1.140503  ])

In [22]:
coeficientes

array([[ 0.15781363, -0.27344866,  0.46200619,  0.05228046,  0.29359627,
         0.13286547],
       [ 0.1406199 ,  0.34339677,  0.21356699, -0.31111205, -0.01317586,
         0.14146892],
       [ 0.03300812, -0.11167035,  0.53405101,  0.02652048,  0.39445031,
        -0.11649181],
       [ 0.16968451, -0.44341644, -0.04982164,  0.174108  , -0.08272259,
         0.27936869],
       [ 0.07632275, -0.35876913, -0.05524693,  0.21737314,  0.04842058,
         0.33702221],
       [ 0.18156988, -0.25773287,  0.0403974 ,  0.04213005, -0.33216136,
        -0.13487046],
       [ 0.13351884,  0.23647025,  0.02192922,  0.53039522, -0.00813702,
        -0.03902751],
       [ 0.31566282,  0.02112889, -0.1746476 , -0.11437245,  0.13374722,
        -0.20052496],
       [ 0.17798456,  0.35504461,  0.0127601 ,  0.34655025,  0.05885659,
         0.12314978],
       [ 0.08763937, -0.03852455, -0.1623026 ,  0.04758399, -0.02471281,
        -0.54521937],
       [ 0.13712201, -0.10637166,  0.2073308 ,  0.

In [23]:
Z

array([[-1.12177585, -1.18225141,  3.66193973,  1.08091972,  3.08416096,
        -0.83802181],
       [-1.11468691, -1.18417302,  3.66232928,  1.09073824,  3.07743037,
        -0.83709805],
       [-0.99491941, -1.57357953,  4.69772411,  1.31731134,  3.912684  ,
        -0.94020808],
       ...,
       [-0.7676707 , -3.26765632, -0.98520556,  1.93561005,  0.17248306,
         2.30534729],
       [ 0.35589709,  0.67843536,  1.00293556, -0.66588834, -1.81479136,
        -1.04864227],
       [-0.8668898 ,  1.19845904, -0.19348689, -0.47633782, -0.60088129,
        -0.03119761]])

In [24]:
varianza_explicada

array([0.23692547, 0.11384602, 0.08834337, 0.08170137, 0.07111615,
       0.04958133])

## PCA de scikit-learn


In [25]:
from sklearn.decomposition import PCA

In [26]:
pca = PCA(n_components=6,svd_solver='full')
pca.fit(df_nutrientes)

PCA(copy=True, iterated_power='auto', n_components=6, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [27]:
print(pca.components_)

[[ 0.15781363  0.1406199   0.03300812  0.16968451  0.07632275  0.18156988
   0.13351884  0.31566282  0.17798456  0.08763937  0.13712201  0.28410233
   0.33777895  0.34132464  0.27245318  0.16811197  0.18080591  0.2998569
   0.24134847  0.09356749  0.19940258  0.0923186   0.24355123]
 [-0.27344866  0.34339677 -0.11167035 -0.44341644 -0.35876913 -0.25773287
   0.23647025  0.02112889  0.35504461 -0.03852455 -0.10637166 -0.09709328
   0.08480073  0.07347102 -0.0751503  -0.10517259  0.21266898 -0.09381195
  -0.10336119  0.08878313  0.08744763  0.23932243  0.17779772]
 [-0.46200619 -0.21356699 -0.53405101  0.04982164  0.05524693 -0.0403974
  -0.02192922  0.1746476  -0.0127601   0.1623026  -0.2073308   0.230985
   0.16488461  0.19209808  0.18435137 -0.12813852 -0.15226269  0.08710919
  -0.20122529 -0.0726296  -0.27481414 -0.16336133 -0.03863878]
 [ 0.05228046 -0.31111205  0.02652048  0.174108    0.21737314  0.04213005
   0.53039522 -0.11437245  0.34655025  0.04758399  0.02617371 -0.0324812
  

In [28]:
print(pca.explained_variance_ratio_)

[0.23692547 0.11384602 0.08834337 0.08170137 0.07111615 0.04958133]


In [29]:
pca.singular_values_

array([216.70704815, 150.21941979, 132.32873239, 127.257042  ,
       118.72748693,  99.13482929])

In [30]:
z = pca.transform(df_nutrientes)

In [31]:
print(z)

[[-1.12177585 -1.18225141 -3.66193973  1.08091972 -3.08416096  0.83802181]
 [-1.11468691 -1.18417302 -3.66232928  1.09073824 -3.07743037  0.83709805]
 [-0.99491941 -1.57357953 -4.69772411  1.31731134 -3.912684    0.94020808]
 ...
 [-0.7676707  -3.26765632  0.98520556  1.93561005 -0.17248306 -2.30534729]
 [ 0.35589709  0.67843536 -1.00293556 -0.66588834  1.81479136  1.04864227]
 [-0.8668898   1.19845904  0.19348689 -0.47633782  0.60088129  0.03119761]]
